# Reconciliando e adicionando as pesquisadoras à Wikidata

Reconciliação foi feita através do Google Sheets, usando o add-on "[Wikipedia and Wikidata Tools](https://workspace.google.com/marketplace/app/wikipedia_and_wikidata_tools/595109124715?pann=cwsdp&hl=en)"

Nome completo foi utilizado como input para a função "=WIKIDATASEARCH()" no Sheets, que nos forneceu uma coluna com os identificadores Wikidata para as pesquisadoras (researcherID). 

No caso de uma pesquisadora não já possuir um item na Wikidata, este foi criado manualmente pela [interface web](https://www.wikidata.org/wiki/Special:NewItem), porém outra opção ainda melhor é tentar adicioná-las através do [Author Disambiguator](https://author-disambiguator.toolforge.org/).

Além disso, criamos um item referente ao programa, [Q110097471](https://www.wikidata.org/wiki/Q110097471), que será utilizado na declaração 'Afiliação' ([P1416](https://www.wikidata.org/wiki/Property:P1416)) da pesquisadora.

In [ ]:
from time import gmtime, strftime
import pandas as pd

profs = pd.read_csv("docentes_reconciliados.csv")

# Extraindo apenas o 'identificador' do Lattes, a parte adicionada à Wikidata
profs["lattes_id"] = profs["lattes"].str.replace(
    "http://lattes.cnpq.br/", "", regex=True
    )

Após ler os dados, podemos criar [QuickStatements](https://quickstatements.toolforge.org/), um formato e uma ferramenta para realizar edições em lote na Wikidata. Para isto, iremos iterar sobre cada linha da planilha e criar declarações para cada pesquisadora:
- Um atribuindo uma descrição básica (Den = 'researcher')
- Um atribuindo sua ocupação como Researcher ([Q1650915](https://www.wikidata.org/wiki/Q1650915))
- Um atribuindo sua afiliação ([P1416](https://www.wikidata.org/wiki/Property:P1416)), ao programa de pós-graduação UTFPR/UFPR-CP ([Q110097471](https://www.wikidata.org/wiki/Q110097471)), com uma referência (S854) ao site que raspamos e a data (S813) em que foi adquirida a informação.
- E, por fim, um atribuindo o número do identificador Lattes ([P1007](https://www.wikidata.org/wiki/Property:P1007)) à pesquisadora

Esse nível de informação já é adequado para um item inicial, com informações mais específicas podendo ser adicionadas ao item posteriormente.

In [ ]:
today_date = strftime("+%Y-%m-%dT00:00:00Z/11", gmtime())
ref_url = "http://www.utfpr.edu.br/cursos/coordenacoes/stricto-sensu/ppgab/area-academica/docentes"
grad_program = "Q110097471"

affiliation_ref = f'S854|"{ref_url}"|S813|{today_date}'

with open("profs_QS.qs", "w") as qs:
    for _, row in profs.iterrows():
        
        d_en = f"{row['researcherID']}|Den|\"researcher\"\n"
        occupation = f"{row['researcherID']}|P106|Q1650915\n"

        affiliation = f"{row['researcherID']}|P1416|{grad_program}|P2868|Q121594|"
        lattes = f"{row['researcherID']}|P1007|\"{row['lattes_id']}\"|"
        lattes_ref = f"S854|\"{row['lattes']}\""

        generals = f"{d_en}{occupation}"
        affiliation_statement = f"{affiliation}{affiliation_ref}\n"
        lattes_statement = f"{lattes}{lattes_ref}\n"

        qs.write(
            f"{generals}{affiliation_statement}{lattes_statement}\n"
        )
